In [ ]:
from keras.models.import Sequential
from keras.layers import Dense
import random

class Agent:
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.lr = 0.001
        self.gamma = 0.99
        self.batchsize = 32
        self.epsilon = 0.05
        self.memory_size = 10**4
        self.model = self._build_model()
        self.memory = []
        
    def _build_model(self):
        model = Sequential()
        model.add( Dense(self.hidden_dim, input_dim = self.input_dim, activation = 'relu') )
        model.add( Dense(self.hidden_dim, activation = 'relu') )
        model.add( Dense(self.output_dim, activation = 'linear') )  #Q function
        return model
    
    def act(self,state):
        
        #Do epsilon greedy
        temp = np.random.rand()
        if temp <= self.epsilon:
            action = np.random.choice(range(self.output_dim))
            return action
        else:
            Qs = self.model.predict(state)[0]
            action = np.argmax(Qs)
            return action
        
        
    def remember(self,event):
        # (S,A,R,S1,D) = event
        # D = done
        if self.memory < self.memory_size
            self.memory.append(event)
        else:
            self.memory[0] = event
            
            
    def learn(self):
        """
        Do experience replay
        
        Loss = E_t ( Q(s_t, a_t) - y_t^2  )^2
        
        where y_t = r + (1-D)*gamma*max*Q_target(s1_t)
        
        So I go over the target network
        
        """
        
        #Grab batch
        (S,A,R,S1,D) = random.sample(self.memory, self.batchsize)
        
        #Find y_i
        Q_target_next_vec = self.target_model.predict(S1)     #(N_samples, dim_action)
        Q_target_next_max = max(Q_target_next_vec, axis = 1)  #(N_samples)
        yi = R + (1-D)*self.gamma*Q_target_next_max           #(N_samples)
        
        #Setup Q_want
        Q = self.model.predict(S)
        
        #All elements of Q, except the one corresponding to the chosen action
        #Stay zero
        
        